In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

sys.path.append(str(Path("..").resolve()))
from config.settings import DATA_RAW, DATA_PROCESSED, RANDOM_STATE, SMMLV

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.4f}".format)

print("✅ Imports OK")

✅ Imports OK


In [2]:
df = pd.read_parquet(DATA_PROCESSED / "secop_eda.parquet")
print(f"Loaded {len(df):,} rows × {df.shape[1]} columns")

# Confirm key columns exist
required = [
    "id_contrato", "codigo_entidad", "codigo_proveedor",
    "valor_del_contrato", "fecha_de_inicio_del_contrato",
    "fecha_de_fin_del_contrato", "fecha_de_firma",
    "modalidad_de_contratacion", "estado_contrato",
    "dias_adicionados", "sector", "departamento"
]
missing = [c for c in required if c not in df.columns]
print(f"Missing required columns: {missing if missing else 'None ✅'}")

Loaded 1,475,699 rows × 35 columns
Missing required columns: None ✅


In [3]:
# ── Temporal Features ───────────────────────────────────────────
feat = df[["id_contrato", "codigo_entidad", "codigo_proveedor",
           "valor_del_contrato", "fecha_de_inicio_del_contrato",
           "fecha_de_fin_del_contrato", "fecha_de_firma",
           "modalidad_de_contratacion", "estado_contrato",
           "dias_adicionados", "sector", "departamento",
           "year", "month", "quarter"]].copy()

# Contract duration in days
feat["duracion_dias"] = (
    feat["fecha_de_fin_del_contrato"] - feat["fecha_de_inicio_del_contrato"]
).dt.days

# Days from signature to contract start (rush indicator)
feat["dias_firma_a_inicio"] = (
    feat["fecha_de_inicio_del_contrato"] - feat["fecha_de_firma"]
).dt.days

# Flag implausibly short signature windows (same day or next day)
feat["flag_rush"] = (
    feat["dias_firma_a_inicio"].between(0, 1, inclusive="both")
).astype(int)

# End of fiscal year pressure flags
feat["flag_q4"] = (feat["quarter"] == 4).astype(int)
feat["flag_december"] = (feat["month"] == 12).astype(int)

# Very short contracts (under 30 days) — often used to avoid oversight
feat["flag_short_contract"] = (feat["duracion_dias"] < 30).astype(int)

# Very long contracts (over 2 years)
feat["flag_long_contract"] = (feat["duracion_dias"] > 730).astype(int)

# Log contract value
feat["log_valor"] = np.log10(feat["valor_del_contrato"].clip(lower=1))

print("Temporal features created:")
temporal_cols = ["duracion_dias", "dias_firma_a_inicio", "flag_rush",
                 "flag_q4", "flag_december", "flag_short_contract",
                 "flag_long_contract", "log_valor"]
print(feat[temporal_cols].describe().round(2).to_string())

Temporal features created:
       duracion_dias  dias_firma_a_inicio      flag_rush        flag_q4  flag_december  flag_short_contract  flag_long_contract      log_valor
count 1,475,690.0000       1,464,367.0000 1,475,699.0000 1,475,699.0000 1,475,699.0000       1,475,699.0000      1,475,699.0000 1,475,699.0000
mean        207.7600               2.0000         0.4700         0.1400         0.0400               0.0500              0.0100         7.3700
std         164.3800              25.6700         0.5000         0.3500         0.2000               0.2300              0.0700         0.5600
min        -274.0000          -1,901.0000         0.0000         0.0000         0.0000               0.0000              0.0000         0.0000
25%         105.0000               0.0000         0.0000         0.0000         0.0000               0.0000              0.0000         7.0400
50%         180.0000               1.0000         0.0000         0.0000         0.0000               0.0000        

In [4]:
# ── Contract-level flags ─────────────────────────────────────────

# Direct award flag
direct_keywords = ["directa", "régimen especial"]
feat["is_direct"] = feat["modalidad_de_contratacion"].str.lower().str.contains(
    "|".join(direct_keywords), na=False
).astype(int)

# Modified contract flag
feat["is_modified"] = (feat["estado_contrato"] == "Modificado").astype(int)

# Cancelled contract flag
feat["is_cancelled"] = (feat["estado_contrato"] == "Cancelado").astype(int)

# Days added (contract extension amount)
feat["dias_adicionados"] = pd.to_numeric(feat["dias_adicionados"], errors="coerce").fillna(0)
feat["flag_extended"] = (feat["dias_adicionados"] > 0).astype(int)

# Extreme value flag (above 99.9th percentile — likely data error)
cap = feat["valor_del_contrato"].quantile(0.999)
feat["flag_extreme_value"] = (feat["valor_del_contrato"] > cap).astype(int)

print("Contract flags:")
flag_cols = ["is_direct", "is_modified", "is_cancelled", "flag_extended", "flag_extreme_value"]
print(feat[flag_cols].mean().mul(100).round(2).to_string())
print("\n(values are % of contracts with flag=1)")

Contract flags:
is_direct            90.5800
is_modified          17.7400
is_cancelled          0.1100
flag_extended        11.3300
flag_extreme_value    0.1000

(values are % of contracts with flag=1)


In [5]:
# ── Vendor-level features ────────────────────────────────────────
# Computed on FULL dataset then joined back
# These capture each vendor's historical behavior pattern

print("Computing vendor features...")

vendor_agg = df.groupby("codigo_proveedor").agg(
    vendor_total_contracts=("id_contrato", "count"),
    vendor_total_spend=("valor_del_contrato", "sum"),
    vendor_mean_value=("valor_del_contrato", "mean"),
    vendor_median_value=("valor_del_contrato", "median"),
    vendor_distinct_agencies=("codigo_entidad", "nunique"),
    vendor_direct_rate=("is_direct", "mean"),
    vendor_modified_rate=("is_modified", "mean"),
    vendor_first_contract=("fecha_de_inicio_del_contrato", "min"),
    vendor_last_contract=("fecha_de_inicio_del_contrato", "max"),
).reset_index()

# Vendor tenure in days
vendor_agg["vendor_tenure_days"] = (
    vendor_agg["vendor_last_contract"] - vendor_agg["vendor_first_contract"]
).dt.days.clip(lower=0)

# Log transforms for skewed distributions
vendor_agg["log_vendor_total_spend"] = np.log10(vendor_agg["vendor_total_spend"].clip(lower=1))
vendor_agg["log_vendor_mean_value"] = np.log10(vendor_agg["vendor_mean_value"].clip(lower=1))

# Agency diversity score — vendors working with very few agencies are higher risk
# (normalized: 1 = max diversity, 0 = single agency)
max_agencies = vendor_agg["vendor_distinct_agencies"].max()
vendor_agg["vendor_agency_diversity"] = vendor_agg["vendor_distinct_agencies"] / max_agencies

# Drop date columns before joining
vendor_agg = vendor_agg.drop(columns=["vendor_first_contract", "vendor_last_contract"])

# Join to feature matrix
feat = feat.merge(vendor_agg, on="codigo_proveedor", how="left")

print(f"Vendor features added. Sample:")
vendor_feat_cols = ["vendor_total_contracts", "vendor_distinct_agencies",
                    "vendor_direct_rate", "vendor_agency_diversity", "vendor_tenure_days"]
print(feat[vendor_feat_cols].describe().round(2).to_string())

Computing vendor features...
Vendor features added. Sample:
       vendor_total_contracts  vendor_distinct_agencies  vendor_direct_rate  vendor_agency_diversity  vendor_tenure_days
count          1,475,699.0000            1,475,699.0000      1,475,699.0000           1,475,699.0000      1,475,699.0000
mean                   9.7500                    4.2600              0.9100                   0.0100            574.1000
std                   42.5000                   20.3100              0.2800                   0.0500            407.5700
min                    1.0000                    1.0000              0.0000                   0.0000              0.0000
25%                    2.0000                    1.0000              1.0000                   0.0000            224.0000
50%                    3.0000                    1.0000              1.0000                   0.0000            539.0000
75%                    5.0000                    2.0000              1.0000                  

In [6]:
# ── Agency-level features ────────────────────────────────────────
print("Computing agency features...")

# HHI — Herfindahl-Hirschman Index of vendor spend concentration per agency
# HHI = sum of squared market shares; range [0,1]; higher = more concentrated
def compute_hhi(series):
    total = series.sum()
    if total == 0:
        return 0
    shares = series / total
    return (shares ** 2).sum()

agency_hhi = df.groupby("codigo_entidad")["valor_del_contrato"].apply(
    lambda x: compute_hhi(
        df.loc[x.index].groupby("codigo_proveedor")["valor_del_contrato"].sum()
    )
).reset_index()
agency_hhi.columns = ["codigo_entidad", "agency_hhi"]

# Top vendor share per agency
def top_vendor_share(group):
    vendor_spend = group.groupby("codigo_proveedor")["valor_del_contrato"].sum()
    total = vendor_spend.sum()
    return vendor_spend.max() / total if total > 0 else 0

agency_top_vendor = df.groupby("codigo_entidad").apply(
    top_vendor_share, include_groups=False
).reset_index()
agency_top_vendor.columns = ["codigo_entidad", "agency_top_vendor_share"]

# General agency stats
agency_agg = df.groupby("codigo_entidad").agg(
    agency_total_contracts=("id_contrato", "count"),
    agency_total_spend=("valor_del_contrato", "sum"),
    agency_direct_rate=("is_direct", "mean"),
    agency_modified_rate=("is_modified", "mean"),
    agency_distinct_vendors=("codigo_proveedor", "nunique"),
    agency_median_value=("valor_del_contrato", "median"),
).reset_index()

# Merge agency features together
agency_feat = agency_agg.merge(agency_hhi, on="codigo_entidad", how="left")
agency_feat = agency_feat.merge(agency_top_vendor, on="codigo_entidad", how="left")

# Flag highly concentrated agencies
agency_feat["flag_agency_concentrated"] = (
    agency_feat["agency_top_vendor_share"] > 0.5
).astype(int)

# Join to feature matrix
feat = feat.merge(agency_feat, on="codigo_entidad", how="left")

print("Agency features added.")
print(f"  Agencies with >50% top vendor concentration: "
      f"{agency_feat['flag_agency_concentrated'].sum():,} "
      f"({agency_feat['flag_agency_concentrated'].mean()*100:.1f}%)")
print(f"  Mean agency HHI: {agency_feat['agency_hhi'].mean():.3f}")
print(f"  Mean top vendor share: {agency_feat['agency_top_vendor_share'].mean()*100:.1f}%")

Computing agency features...
Agency features added.
  Agencies with >50% top vendor concentration: 684 (29.0%)
  Mean agency HHI: 0.285
  Mean top vendor share: 37.7%


In [7]:
# ── Proxy Labels ─────────────────────────────────────────────────
# These are NEVER called fraud labels. They are auditor-endorsed
# risk signals validated in procurement literature.

# Strong proxy — used for model calibration
# direct award + modified = structurally high-risk combination
feat["proxy_strong"] = (
    (feat["is_direct"] == 1) & (feat["is_modified"] == 1)
).astype(int)

# Medium proxy — used for validation only
feat["proxy_medium"] = (
    (feat["is_direct"] == 1) |
    (feat["is_modified"] == 1) |
    (feat["flag_rush"] == 1)
).astype(int)

strong_rate = feat["proxy_strong"].mean() * 100
medium_rate = feat["proxy_medium"].mean() * 100

print("Proxy Label Summary:")
print(f"  Strong proxy (direct AND modified): {feat['proxy_strong'].sum():,} ({strong_rate:.1f}%)")
print(f"  Medium proxy (direct OR modified OR rush): {feat['proxy_medium'].sum():,} ({medium_rate:.1f}%)")
print(f"\n  Strong proxy base rate {strong_rate:.1f}% is within the")
print(f"  target range for calibration (8-20%).")

Proxy Label Summary:
  Strong proxy (direct AND modified): 235,175 (15.9%)
  Medium proxy (direct OR modified OR rush): 1,399,948 (94.9%)

  Strong proxy base rate 15.9% is within the
  target range for calibration (8-20%).


In [8]:
# Add nombre_entidad back — it was dropped during feature engineering merges
# Pull it directly from the raw EDA dataframe
nombre_map = pd.read_parquet(DATA_PROCESSED / "secop_eda.parquet")[
    ["id_contrato", "nombre_entidad"]
].drop_duplicates("id_contrato")

feat = feat.merge(nombre_map, on="id_contrato", how="left")

print(f"nombre_entidad added: {feat['nombre_entidad'].notna().sum():,} non-null values")

nombre_entidad added: 1,475,699 non-null values


In [9]:
# ── Assemble final feature matrix ────────────────────────────────

# Columns to keep — everything the models will see
FEATURE_COLS = [
    # Identifiers (not used in models, kept for joining)
    "id_contrato", "codigo_entidad", "codigo_proveedor",
    "nombre_entidad", "year", "month", "quarter", "departamento", "sector",

    # Raw value
    "valor_del_contrato", "log_valor",

    # Temporal features
    "duracion_dias", "dias_firma_a_inicio",
    "flag_rush", "flag_q4", "flag_december",
    "flag_short_contract", "flag_long_contract",

    # Contract flags
    "is_direct", "is_modified", "is_cancelled",
    "flag_extended", "flag_extreme_value",
    "dias_adicionados",

    # Vendor features
    "vendor_total_contracts", "vendor_total_spend",
    "vendor_mean_value", "vendor_median_value",
    "vendor_distinct_agencies", "vendor_direct_rate",
    "vendor_modified_rate", "vendor_tenure_days",
    "vendor_agency_diversity", "log_vendor_total_spend",
    "log_vendor_mean_value",

    # Agency features
    "agency_total_contracts", "agency_total_spend",
    "agency_direct_rate", "agency_modified_rate",
    "agency_distinct_vendors", "agency_median_value",
    "agency_hhi", "agency_top_vendor_share",
    "flag_agency_concentrated",

    # Proxy labels
    "proxy_strong", "proxy_medium",
]

# Keep only columns that exist (safety check)
available = [c for c in FEATURE_COLS if c in feat.columns]
missing_cols = [c for c in FEATURE_COLS if c not in feat.columns]

if missing_cols:
    print(f"⚠️  Missing columns: {missing_cols}")

feature_matrix = feat[available].copy()

print(f"\nFeature matrix shape: {feature_matrix.shape}")
print(f"Features available:   {len(available)}")
print(f"\nNull counts in feature matrix:")
nulls = feature_matrix.isnull().sum()
nulls = nulls[nulls > 0]
print(nulls.to_string() if len(nulls) > 0 else "  None ✅")


Feature matrix shape: (1475699, 46)
Features available:   46

Null counts in feature matrix:
duracion_dias              9
dias_firma_a_inicio    11332


In [10]:
# ── Null handling ────────────────────────────────────────────────

# duracion_dias and dias_firma_a_inicio are null when dates are missing
# Fill with median — defensible for temporal features
for col in ["duracion_dias", "dias_firma_a_inicio"]:
    if col in feature_matrix.columns:
        median_val = feature_matrix[col].median()
        feature_matrix[col] = feature_matrix[col].fillna(median_val)
        print(f"  Filled {col} nulls with median: {median_val:.0f}")

# Final null check
remaining_nulls = feature_matrix.isnull().sum().sum()
print(f"\nRemaining nulls after fill: {remaining_nulls}")

# Save
output_path = DATA_PROCESSED / "feature_matrix.parquet"
feature_matrix.to_parquet(output_path, index=False, compression="snappy")

print(f"\n{'='*55}")
print(f"✅ FEATURE ENGINEERING COMPLETE")
print(f"{'='*55}")
print(f"  Rows:              {len(feature_matrix):,}")
print(f"  Features:          {len(feature_matrix.columns)}")
print(f"  Strong proxy rate: {feature_matrix['proxy_strong'].mean()*100:.1f}%")
print(f"  Medium proxy rate: {feature_matrix['proxy_medium'].mean()*100:.1f}%")
print(f"  Saved to:          {output_path}")
print(f"{'='*55}")

  Filled duracion_dias nulls with median: 180
  Filled dias_firma_a_inicio nulls with median: 1

Remaining nulls after fill: 0

✅ FEATURE ENGINEERING COMPLETE
  Rows:              1,475,699
  Features:          46
  Strong proxy rate: 15.9%
  Medium proxy rate: 94.9%
  Saved to:          C:\Users\HP\Collections\VsCode Projects\AuditLens\data\processed\feature_matrix.parquet
